In [1]:
from pandora_bayesopt.test_functions.robot_pushing.robot_pushing import robot_pushing_14d

pygame 2.1.0 (SDL 2.0.16, Python 3.9.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import torch
from torch import Tensor
import numpy as np
from botorch.utils.sampling import draw_sobol_samples
from botorch.acquisition import ExpectedImprovement
from pandora_bayesopt.acquisition.ei_puc import ExpectedImprovementWithCost
from pandora_bayesopt.acquisition.gittins import GittinsIndex
from pandora_bayesopt.acquisition.budgeted_multi_step_ei import BudgetedMultiStepLookaheadEI
from pandora_bayesopt.bayesianoptimizer import BayesianOptimizer

In [3]:
# use a GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set default tensor type to float64
torch.set_default_dtype(torch.float64)

In [4]:
seed = 42
torch.manual_seed(seed)

In [5]:
target_location = 10.0 * torch.rand([2]) - 5.0
target_location2 = 10.0 * torch.rand([2]) - 5.0

In [6]:
target_location, target_location2

(tensor([-4.4185, -4.3709]), tensor([-3.7641, -4.4742]))

In [7]:
init_location = torch.tensor([0, 2])
init_location2 = torch.tensor([0, -2])

In [37]:
def unnorm_X(X: torch.Tensor) -> torch.Tensor:
    X_unnorm = X.clone()
    # Check if the tensor is higher than 3-dimensional
    if X.dim() > 3:
        # Assuming the extra unwanted dimension is at position 1 (the second position)
        X_unnorm = X_unnorm.view(-1, X.size(-1))  # Remove the singleton dimension
    # Check the dimensionality of X and adjust accordingly
    if X.dim() == 3:
        # Remove the singleton dimension assuming it's the second one
        X_unnorm = X_unnorm.squeeze(1)
    elif X.dim() == 1:
        # If 1-dimensional, add a dimension to make it 2D (e.g., for batch size of 1)
        X_unnorm = X_unnorm.unsqueeze(0)
    X_unnorm[:, :2] = 10.0 * X_unnorm[:, :2] - 5.0
    X_unnorm[:, 2:4] = 5 * X_unnorm[:, 2:4]
    X_unnorm[:, 4] = 29.0 * X_unnorm[:, 4] + 1.0
    X_unnorm[:, 5] = 2 * np.pi * X_unnorm[:, 5]
    X_unnorm[:, 6:8] = 10.0 * X_unnorm[:, 6:8] - 5.0
    X_unnorm[:, 8:10] = 5 * X_unnorm[:, 8:10]
    X_unnorm[:, 10] = 29 * X_unnorm[:, 10] + 1.0
    X_unnorm[:, 11] = 2 * np.pi * X_unnorm[:, 11]
    X_unnorm[:, 12:] = 2 * np.pi * X_unnorm[:, 12:]

    return X_unnorm

def objective_cost_function(X: torch.Tensor) -> torch.Tensor:
    X_unnorm = unnorm_X(X)
    objective_X = []
    cost_X = []
    
    for x in X_unnorm:
        np.random.seed(0)
        object_location, object_location2, robot_location, robot_location2 = torch.tensor(robot_pushing_14d(x[0].item(), x[1].item(), x[2].item(), x[3].item(), x[4].item(), x[5].item(), x[6].item(), x[7].item(), x[8].item(), x[9].item(), x[10].item(), x[11].item(), x[12].item(), x[13].item()))
        objective_X.append(-torch.dist(target_location, object_location)-torch.dist(target_location2, object_location2))
        moving_distance = torch.dist(x[:2], robot_location)+torch.dist(x[6:8], robot_location2)+0.1
        
        cost_X.append(moving_distance)

    np.random.seed()  # Reset the seed
    
    objective_X = torch.tensor(objective_X)
    cost_X = torch.tensor(cost_X)
    return objective_X, cost_X

In [9]:
seed = 1
torch.manual_seed(seed)

In [10]:
dim = 14
bounds = torch.stack([torch.zeros(dim), torch.ones(dim)])
init_x = draw_sobol_samples(bounds=bounds, n=1, q=2*(dim+1)).squeeze(0)

In [16]:
objective_X, cost_X = objective_cost_function(init_x)

In [17]:
objective_X, cost_X

(tensor([-14.7659, -12.2576, -12.2576, -12.2576, -12.2576, -13.4437, -12.2576,
         -12.2576, -12.2576, -12.2576, -12.2576, -12.2576, -14.0308, -12.2576,
         -12.3664, -12.6729, -12.3685, -12.2576, -12.8875, -12.2576, -13.5099,
         -12.2576, -12.7769, -12.3122, -12.2576, -12.2576, -12.2576, -11.9910,
         -11.9794, -12.2576]),
 tensor([14.9645, 13.3399, 11.6955,  5.9913, 14.2644, 18.1455, 17.1731,  7.5569,
         14.9609,  5.4775,  4.7969, 17.1050, 15.3101, 14.0806,  8.7827,  9.3806,
         12.3590,  7.4306,  2.9377,  6.3146,  8.5321, 16.2050,  5.2135, 15.5288,
         17.5221,  6.4471, 16.4901, 16.5528, 11.3678,  8.4962]))

In [18]:
maximize = True
input_standardize = True
budget = 20
Optimizer = BayesianOptimizer(
        objective_cost=objective_cost_function, 
        dim=dim, 
        maximize=maximize, 
        initial_points=init_x, 
        input_standardize=input_standardize
    )
Optimizer.run_until_budget(
            budget=budget,
            acquisition_function_class = ExpectedImprovementWithCost
        )

Iteration 0, New point: [0.57842758 0.87961768 0.26781945 0.36382642 0.73224356 0.38738398
 0.54794989 0.67008403 0.33032038 0.86813636 0.06791416 0.19406101
 0.75371731 0.62978602], New value: -12.207867930901958
Best observed value: -11.979440263560832
Current acquisition value: 0.013591923351439997
Cumulative cost: 5.406435062048684
Running time: 8.181542999999998

Iteration 1, New point: [0.56135752 0.77866619 0.27742945 0.34275905 0.65971034 0.55693654
 0.61282477 0.67557888 0.4010546  0.79513321 0.09932942 0.30696246
 0.46986676 0.71806048], New value: -12.159645052713874
Best observed value: -11.979440263560832
Current acquisition value: 0.014211666627187692
Cumulative cost: 11.211253777605517
Running time: 6.936563999999999

Iteration 2, New point: [0.46030229 0.80841257 0.29883932 0.50290263 0.43872943 0.52804327
 0.47924061 0.6098113  0.28342876 0.79739921 0.07519244 0.29477269
 0.57616047 0.47218064], New value: -12.987528597786426
Best observed value: -11.979440263560832
Cu

In [19]:
maximize = True
input_standardize = True
budget = 20
Optimizer = BayesianOptimizer(
        objective_cost=objective_cost_function, 
        dim=dim, 
        maximize=maximize, 
        initial_points=init_x, 
        input_standardize=input_standardize
    )
Optimizer.run_until_budget(
            budget=budget,
            acquisition_function_class = GittinsIndex,
            lmbda = 0.0001
        )

Iteration 0, New point: [0.56422224 0.67087796 0.34145352 0.65339473 0.27987554 0.7885727
 0.49838885 0.69310072 0.35522463 0.78513832 0.25163533 0.46447239
 0.12382329 0.30176179], New value: -14.056389940631117
Best observed value: -11.979440263560832
Current acquisition value: -10.769536565620601
Cumulative cost: 5.0301266346352875
Running time: 18.603854

Iteration 1, New point: [0.64864001 0.75311841 0.27029397 0.40734736 0.64476262 0.36764584
 0.56944918 0.55067039 0.35569456 0.85066346 0.08952755 0.17397713
 0.92756943 0.74561181], New value: -12.249035999170989
Best observed value: -11.979440263560832
Current acquisition value: -10.652211572468318
Cumulative cost: 10.58044120187924
Running time: 22.491062

Iteration 2, New point: [0.31676408 0.94871247 0.25982836 0.31000382 0.71358482 0.39847665
 0.56863652 0.66402787 0.32883151 0.85991612 0.07219499 0.15998541
 0.80158578 0.73204078], New value: -12.248238729283987
Best observed value: -11.979440263560832
Current acquisition v